In [1]:
import pandas as pd

In [2]:
# importing book sales data

df_booksales = pd.read_excel(f'./Challenge 427/Book Sales.xlsx')

In [3]:
df_booksales.head()

,Date,BookTitle
0,2024-01-01,Lord of the Flies
1,2024-01-01,1984
2,2024-01-01,Lord of the Flies
3,2024-01-01,1984
4,2024-01-01,The Great Gatsby


In [4]:
# importing analysis refernence period

df_analysisperiod = pd.read_excel('./Challenge 427/Analysis Period.xlsx')

In [5]:
df_analysisperiod.head()

,Start_YMD,End_YMD,MonthToReport
0,2023-12-01,2024-05-31,1
1,2023-11-01,2024-04-30,2
2,2023-10-01,2024-03-31,3


In [8]:
# generating records between start ymd and end ymd

date_scaffold = []

for index, row in df_analysisperiod.iterrows():
    start_date = row['Start_YMD']
    end_date = row['End_YMD']
    current_date = row['Start_YMD']
    while current_date <= end_date:
        date_scaffold.append({'initial_date' : start_date, 'scaffold' : current_date , 'end_date' : end_date})
        current_date += pd.DateOffset(days = 1)
        
        
date_scaffold
        
    

[{'initial_date': Timestamp('2023-12-01 00:00:00'),
  'scaffold': Timestamp('2023-12-01 00:00:00'),
  'end_date': Timestamp('2024-05-31 00:00:00')},
 {'initial_date': Timestamp('2023-12-01 00:00:00'),
  'scaffold': Timestamp('2023-12-02 00:00:00'),
  'end_date': Timestamp('2024-05-31 00:00:00')},
 {'initial_date': Timestamp('2023-12-01 00:00:00'),
  'scaffold': Timestamp('2023-12-03 00:00:00'),
  'end_date': Timestamp('2024-05-31 00:00:00')},
 {'initial_date': Timestamp('2023-12-01 00:00:00'),
  'scaffold': Timestamp('2023-12-04 00:00:00'),
  'end_date': Timestamp('2024-05-31 00:00:00')},
 {'initial_date': Timestamp('2023-12-01 00:00:00'),
  'scaffold': Timestamp('2023-12-05 00:00:00'),
  'end_date': Timestamp('2024-05-31 00:00:00')},
 {'initial_date': Timestamp('2023-12-01 00:00:00'),
  'scaffold': Timestamp('2023-12-06 00:00:00'),
  'end_date': Timestamp('2024-05-31 00:00:00')},
 {'initial_date': Timestamp('2023-12-01 00:00:00'),
  'scaffold': Timestamp('2023-12-07 00:00:00'),
  'end

In [9]:
# converting scaffold to a dataframe

df_scaffold = pd.DataFrame(date_scaffold)

df_scaffold

,initial_date,scaffold,end_date
0,2023-12-01,2023-12-01,2024-05-31
1,2023-12-01,2023-12-02,2024-05-31
2,2023-12-01,2023-12-03,2024-05-31
3,2023-12-01,2023-12-04,2024-05-31
4,2023-12-01,2023-12-05,2024-05-31
...,...,...,...
543,2023-10-01,2024-03-27,2024-03-31
544,2023-10-01,2024-03-28,2024-03-31
545,2023-10-01,2024-03-29,2024-03-31
546,2023-10-01,2024-03-30,2024-03-31


In [10]:
# joining scaffold with book sales

df_merge = df_scaffold.merge(df_booksales, left_on = 'scaffold', right_on = 'Date')

df_merge

,initial_date,scaffold,end_date,Date,BookTitle
0,2023-12-01,2023-12-01,2024-05-31,2023-12-01,The Lord of the Rings
1,2023-12-01,2023-12-01,2024-05-31,2023-12-01,1984
2,2023-12-01,2023-12-01,2024-05-31,2023-12-01,1984
3,2023-12-01,2023-12-01,2024-05-31,2023-12-01,Lord of the Flies
4,2023-12-01,2023-12-01,2024-05-31,2023-12-01,Crime and Punishment
...,...,...,...,...,...
91009,2023-10-01,2023-10-31,2024-03-31,2023-10-31,1984
91010,2023-10-01,2023-10-31,2024-03-31,2023-10-31,The Catcher in the Rye
91011,2023-10-01,2023-10-31,2024-03-31,2023-10-31,To Kil A Mockingbird
91012,2023-10-01,2023-10-31,2024-03-31,2023-10-31,The Catcher in the Rye


In [11]:
# finding total sales per analysis period

df_sales = df_merge.groupby('initial_date')['initial_date'].count().reset_index(name = 'total_sales')

df_sales

,initial_date,total_sales
0,2023-10-01,29175
1,2023-11-01,30441
2,2023-12-01,31398


In [12]:
# finding the most popular book sales per analysis period

df_sales_period = df_merge.groupby(['initial_date', 'BookTitle'])['BookTitle'].count().reset_index(name = 'sales')

df_sales_period = df_sales_period.groupby('initial_date').apply(lambda x : x.nlargest(1, 'sales')).reset_index(drop = True)

In [13]:
# joining df_sales_period with df_sales

df_sales.merge(df_sales_period, on = 'initial_date')

,initial_date,total_sales,BookTitle,sales
0,2023-10-01,29175,The Corrections,3689
1,2023-11-01,30441,The Great Gatsby,3873
2,2023-12-01,31398,Lord of the Flies,3993
